<font size="+1" color="#108fff" face="Arial">1. Создать код на Python, который загрузит на локальный компьютер данные о котировках ценных бумаг из списка SnP500</font> 

In [177]:
#Базовые библиотеки
!pip install lxml -qq
import pandas as pd
import numpy as np
from datetime import datetime

<font color="#108fff" face="Arial">1.1. Получить список акций, входящих в SPY в текущий момент. Использована Wikipedia. 
<br>Варианты: сайт S&P или анализ состава ETF SPY</font>

In [4]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', match='Symbol')[0]
tickers = tickers.iloc [:, 0:1] 
tickers = tickers.dropna()
#tickers.head()

<font color="#108fff" face="Arial">1.2. Получить исторические данные по акциям, входящим в SPY</font>

In [6]:
!pip install yfinance -qq
import yfinance as yf

<font color="red" face="Arial"> ! В Yahoo Finance тикер у Berkshire Hathaway Inc. отличается от представленного в списке Wiki. <br>! Перед загрузкой котировок необходимы преобразования.</font>

In [8]:
tickers[tickers['Symbol'] == 'BRK.B'] = 'BRK-B'

In [113]:
Stdate = datetime(2024, 8, 3)
Edate = datetime(2024, 8, 23)
period = '5d'
stocks = yf.download(tickers.Symbol.to_list(), Stdate, Edate, period = period)['Close']
stocks.info()

[*********************100%***********************]  503 of 503 completed

1 Failed download:
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2024-08-03 00:00:00 -> 2024-08-23 00:00:00)')


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 14 entries, 2024-08-05 00:00:00+00:00 to 2024-08-22 00:00:00+00:00
Columns: 503 entries, A to ZTS
dtypes: float64(503)
memory usage: 55.1 KB


<font color="#108fff" face="Arial">1.3. Обработать пропуски.</font>

In [121]:
#Установить порог данных с Null
threshold = len(stocks) * 0.25
#Удалить колонки за порогом
stocks = stocks.dropna(axis=1, thresh=threshold)
#Заменить пропуски последним значением
stocks = stocks.ffill()
#Заменить пропуски ближайшим следующим значением
stocks = stocks.bfill()

<font color="#108fff" face="Arial">1.4. Преобразовать формат даты.</font>

In [127]:
stocks.index = pd.to_datetime(stocks.index).strftime('%Y-%m-%d')

<font color="#108fff" face="Arial">1.4. Выгрузить данные в файл.</font>

In [129]:
filename = f'SPY_stocks_{period}_{Stdate.strftime("%Y%m%d")}_{Edate.strftime("%Y%m%d")}.csv'
stocks.to_csv(filename, header=True, index=True)

<font size="+1" color="#108fff" face="Arial">2. Подготовить таблицы и графики для исследования данных.</font>

<font color="#108fff" face="Arial">2.1. Общая таблица данных с поиском по тикеру.</font>

In [290]:
!pip install itables -qq
from itables import show
show(stocks.transpose())

<font color="#108fff" face="Arial">2.2. Общая таблица данных по доходности с цветовой индикацией.</font>

In [305]:
returns = stocks.pct_change(periods = 2).dropna()
# Define a function to color negative values
def color_negative(v):
    if v < -0.01:
        return 'color: red; background-color: #fceeee;'
    elif v < 0:
        return 'color: red; background-color: #ffffff;'
    elif v > 0.05:
        return 'color: green; background-color: #f2fcf4;'
    else:
        return 'color: black; background-color: #ffffff;'
# Apply the styling function using map
returns.style.map(color_negative).format("{:.2%}")

<font color="#108fff" face="Arial">2.3. Отобрать интересующие тикеры.</font>

In [303]:
#filtered_df = data.iloc[:,12:36]
filtered_df = stocks.loc[:, ['AAPL','MSFT','AMZN','NVDA','GOOGL','TSLA','GOOG','BRK-B','META','UNH','XOM','LLY']]

<font color="#108fff" face="Arial">2.4. Автоматические отчеты по выбранным акциям.</font>

In [299]:
!pip install sweetviz -qq
import sweetviz as sv

In [307]:
report = sv.analyze(filtered_df)
report.show_html('top12_sweetviz_report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report top12_sweetviz_report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [309]:
!pip install dtale -qq
import dtale
d = dtale.show(filtered_df)
d.open_browser()